In [1]:
# !pip install spotipy --upgrade
# !pip install pulp
import spotipy
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import pandas as pd
from IPython.display import clear_output
from pulp import *
import os
# os.chdir("C:\\Users\\User\\OneDrive - Fundacao Getulio Vargas - FGV\\7o Período\\PLI\\spotify_optimal_subplaylist")
from secrets import secrets

In [7]:
# Creating Spotify Connection

## If you want to reproduce this step subscribe to 
## https://developer.spotify.com/dashboard/login, create your app, 
## get your credentials, and then set it as environment variables
cid = secrets['SPOTIPY_CLIEND_ID']
secret = secrets['SPOTIPY_CLIENT_SECRET']

client_credentials_manager = SpotifyClientCredentials(client_id=cid,client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [8]:
# Functon to iterate over paginated results
def get_playlist_tracks(playlist_id):
    results = sp.playlist(playlist_id)['tracks']
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [9]:
tracks = get_playlist_tracks(playlist_id='5TUxgTIxzLbLVh7RUf9V8i')
print("ok")

ok


In [14]:
tr = tracks[0]
sp.audio_features(tr['track']['uri'])

[{'danceability': 0.454,
  'energy': 0.91,
  'key': 6,
  'loudness': -7.766,
  'mode': 1,
  'speechiness': 0.0448,
  'acousticness': 0.0866,
  'instrumentalness': 0.0996,
  'liveness': 0.116,
  'valence': 0.629,
  'tempo': 125.116,
  'type': 'audio_features',
  'id': '7o2CTH4ctstm8TNelqjb51',
  'uri': 'spotify:track:7o2CTH4ctstm8TNelqjb51',
  'track_href': 'https://api.spotify.com/v1/tracks/7o2CTH4ctstm8TNelqjb51',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7o2CTH4ctstm8TNelqjb51',
  'duration_ms': 354520,
  'time_signature': 4}]